In [1]:
import math
import time
import argparse
import numpy as np
import tensorflow as tf
from zmq_ops import ZmqOp

In [2]:
# parser = argparse.ArgumentParser(description='Process some integers.')
# parser.add_argument("-n", "--num", type=int, default=100)


# args = parser.parse_args()
# num = args.num
num = 1000

#from tensorcom
class Stats(object):
    def __init__(self):
        self.lo = float('Inf')
        self.hi = -float('Inf')
        self.sx = 0
        self.sx2 = 0
        self.n = 0

    def __iadd__(self, x):
        self.lo = min(self.lo, np.amin(x))
        self.hi = max(self.hi, np.amax(x))
        self.sx += np.sum(x)
        self.sx2 += np.sum(x**2)
        self.n += x.size
        return self

    def summary(self):
        return "[{:.3g} {:.3g}] mean={:.3g} std={:.3g} n={:d}".format(
            self.lo, self.hi,
            self.sx/self.n,
            (self.sx2/self.n - (self.sx/self.n)**2)**.5,
            self.n
        )


shapes = set()
count = 0
total = 0
stats = Stats()
types = [tf.dtypes.as_dtype(np.dtype('float16')), tf.dtypes.as_dtype(np.dtype('int32'))]
zmq_op = ZmqOp(address="zsub://127.0.0.1:7880").pull(types)
sess =  tf.Session()
def load():
    while True:
        outs = sess.run(zmq_op)
        yield outs


In [3]:
count = 0
total = 0

start = time.time()

for i, x in enumerate(load()):    
#     if i > 0 and i % 100 == 0:
#         print (i)
    a = x[0]
    count += 1
    total += len(a)
    shapes.add((str(a.dtype),) + tuple(a.shape))
    stats += a.astype(np.float32)
    if i == num - 1:
        break
finish = time.time()


print("{} batches, {} samples".format(count, total))
print("{:.3g} s per batch, {:.3g} s per sample".format(
    (finish-start)/count, (finish-start)/total))
print("shapes:", shapes)
print(stats.summary())

1000 batches, 32000 samples
0.000473 s per batch, 1.48e-05 s per sample
shapes: {('float16', 32, 1, 28, 28)}
[-0.424 2.82] mean=-0.0463 std=0.948 n=25088000
